In [70]:
import numpy as np
import pandas as pd


path = './data/'
output = './output/'


In [60]:
# start with an effort to figure out if players are returning the next year
# sometimes their name gets inexplicably reversed, in fact most are reversed in 2018
# need column that answers question (did return?)
# merge in player stats

# ONLY NEED TO RUN ONE TIME
# years = list(range(2010, 2020))
# years = list(map(str, years))
# years.reverse()

# for y in years:
#     player_ext = "Players_{}.csv".format(y)
#     players = pd.read_csv(path+player_ext)
#     if years.index(y) != 0:
#         players = pd.merge(players, ny_players, how='left', on=['TeamID','TeamID'])
      
#     players['AltName'] = players['PlayerName'].str.replace('_',' ')
#     players['AltName'] = ['_'.join(s.split()[::-1]) for s in players['AltName']]
    
# #     players that will play the next year (since I'm going backward)
#     ny_players = players.groupby(['TeamID'])['PlayerName'].apply(list).reset_index(name='NY_Players')
#     ny_players = ny_players[['TeamID','NY_Players']]
#     if years.index(y) != 0:
#         players['NY_Players'] = players['NY_Players'].apply(lambda d: d if isinstance(d, list) else [])
#         players['WillReturn'] = players.apply(lambda x: x['PlayerName'] in x['NY_Players'], axis=1)
#         players['WillReturn2'] = players.apply(lambda x: x['AltName'] in x['NY_Players'], axis=1)
#         players['Returning'] = np.where((players['WillReturn']|players['WillReturn2']), True, False)
#         players = players[['PlayerID','Season','TeamID','PlayerName','AltName','Returning']]
#         players = players.loc[players['PlayerName']!='TEAM']
#         players.to_csv(path+'New_Players_{}.csv'.format(y),index=None)

In [15]:
list(events)

['EventID',
 'Season',
 'DayNum',
 'WTeamID',
 'LTeamID',
 'WPoints',
 'LPoints',
 'ElapsedSeconds',
 'EventTeamID',
 'EventPlayerID',
 'EventType']

In [18]:
# make sure score keeping is relatively consistent
# events.groupby(['EventType'])['EventID'].count()

# 2018 numbers
#made2_dunk     18975
# made2_jump     64606
# made2_lay     110309
# made2_tip       4198

# 2010 numbers
# made2_dunk     16065
# made2_jump     69599
# made2_lay      97364
# made2_tip       4416

# seems okay, scoring seems to have gone up over time, or there are more games

EventType
assist        135116
block          36076
foul_pers     196038
foul_tech       1486
made1_free    151741
made2_dunk     16065
made2_jump     69599
made2_lay      97364
made2_tip       4416
made3_jump     65164
miss1_free     59367
miss2_dunk      1829
miss2_jump    128469
miss2_lay      72835
miss2_tip       2447
miss3_jump    125362
reb_dead       32061
reb_def       247105
reb_off       119795
steal          69508
sub_in        325212
sub_out       326376
timeout        36260
timeout_tv     38518
turnover      144421
Name: EventID, dtype: int64

In [19]:
# only use events we care about
# print(list(events.EventType.unique()))

dont_care = ['foul_pers','foul_tech','timeout_tv', 'reb_dead', 'timeout','sub_in', 'sub_out']
# i don't think i'm going to tally missed shots
missed = ['miss2_jump','miss1_free','miss3_jump', 'miss2_lay',  'miss2_dunk', 'miss2_tip']

event_types = ['assist', 'made3_jump', 'turnover',  'reb_off', 'made2_dunk','made2_jump', 'steal', 'made1_free', 'block','reb_def','made2_lay', 'made2_tip']



2502630
1156370


In [63]:
def filter_events(df):
    df = df.loc[df.EventType.isin(event_types)]
    return df



In [64]:
# categories I'm interested in year over yeat
# aggregate event types
rebounds = ['reb_off','reb_def']
# close range 2pts
cr_2pt = ['made2_lay', 'made2_tip', 'made2_dunk']
# mid range 2pts
mr_2pt = ['made2_jump']
# 3 points
lr = ['made3_jump']
# assists, blocks, steals, fts
# 8 total categories

In [65]:
def aggregate_events(df):
    
    df.loc[df.EventType.isin(rebounds), 'EventType'] = 'rebound'
    df.loc[df.EventType.isin(cr_2pt), 'EventType'] = 'cr_2pt'
    
    return df



In [66]:
# can group events by player id

def player_level_events(df):
    df = df.groupby(['EventPlayerID','EventType'])['EventID'].count().unstack(-1).reset_index().fillna(0)
    df = df.rename(columns={'EventPlayerID':'PlayerID'})
    return df


In [81]:
teams_ext = 'Teams.csv'
teams = pd.read_csv(path+teams_ext)


In [71]:
years = list(range(2010, 2019))
years = list(map(str, years))

for y in years:
    player_ext = 'New_Players_{}.csv'.format(y)
    event_ext = 'Events_{}.csv'.format(y)
    events = pd.read_csv(path+event_ext)
    players = pd.read_csv(path+player_ext)
    
    events = filter_events(events)
    events = aggregate_events(events)
    events = player_level_events(events)
    
    players = pd.merge(players, events, how='left', on=['PlayerID','PlayerID'])
    players = players.loc[players.PlayerName != 'TEAM']
    players = players.fillna(0)
    #player level season stats, and if they return
    players.to_csv(output+'PL_Season_Stats_{}.csv'.format(y),index=None)
    
# above code sucks, i want it all in one dataframe.
# will have to edit next year so that I dont have to run the below code

seasons = list(range(2010,2019))
all_df = None
for season in seasons:
    season_path = "PL_Season_Stats_{}.csv".format(season)
    season = pd.read_csv(output+season_path)
    if all_df is not None:
        all_df = pd.concat([all_df,season],axis=0)
    else:
        all_df = season

all_df.to_csv("PL_Returning.csv",index=None)


In [72]:
print(players.head())

   PlayerID  Season  TeamID       PlayerName          AltName  Returning  \
0    648095    2018    1101       COBB_CHASE       CHASE_COBB       True   
1    648096    2018    1101   FARQHAR_HAYDEN   HAYDEN_FARQHAR      False   
2    648097    2018    1101  FARQUHAR_HAYDEN  HAYDEN_FARQUHAR       True   
3    648098    2018    1101     FOSTER_TEVIN     TEVIN_FOSTER      False   
4    648099    2018    1101  FRANKLIN_JAYLEN  JAYLEN_FRANKLIN       True   

   assist  block  cr_2pt  made1_free  made2_jump  made3_jump  rebound  steal  \
0     0.0    0.0     0.0         0.0         0.0         0.0      0.0    0.0   
1     0.0    1.0     0.0         0.0         0.0         1.0      1.0    0.0   
2     9.0    6.0    14.0         4.0         2.0        13.0     51.0   10.0   
3    89.0    4.0    43.0        94.0        18.0        43.0     82.0   38.0   
4   117.0    5.0    85.0        48.0        31.0         4.0    114.0   45.0   

   turnover  
0       0.0  
1       2.0  
2      11.0  
3     

In [82]:
# print(teams.head())
teams = teams[['TeamID','TeamName']]

,TeamID,TeamName
0,1101,Abilene Chr
1,1102,Air Force
2,1103,Akron
3,1104,Alabama
4,1105,Alabama A&M
5,1106,Alabama St
6,1107,Albany NY
7,1108,Alcorn St
8,1109,Alliant Intl
9,1110,American Univ


In [91]:
def get_pct_totals(df):
    
    cols = ['assist','block','cr_2pt','made1_free','made2_jump','made3_jump','rebound','steal','turnover']

   
    for c in cols:
        new_c = 'pct_'+c
        total_c = 'total_'+c
        df[new_c] = np.round(df[c]/df[total_c], 2)
    return df

In [100]:
# team level returning
pl_sum_cols = ['TeamID','total_assist','total_block','total_cr_2pt','total_made1_free','total_made2_jump','total_made3_jump','total_rebound','total_steal','total_turnover']
cols = ['assist','block','cr_2pt','made1_free','made2_jump','made3_jump','rebound','steal','turnover']
pct_cols = ['pct_'+c for c in cols]
pct_cols.insert(0, 'Returning')
pct_cols.insert(0, 'TeamID')

tl_df = None
for y in years:
    pl = pd.read_csv(output+'PL_Season_Stats_{}.csv'.format(y))
    pl = pl.groupby(['TeamID','Returning'])['assist','block','cr_2pt','made1_free','made2_jump','made3_jump','rebound','steal','turnover'].sum().reset_index()
    pl_sum = pl.groupby(['TeamID'])['assist','block','cr_2pt','made1_free','made2_jump','made3_jump','rebound','steal','turnover'].sum().reset_index()
    pl_sum.columns=pl_sum_cols
    pl = pd.merge(pl, pl_sum, how='left',on=['TeamID','TeamID'])
    pl = get_pct_totals(pl)
    pl = pl[pct_cols]
    pl = pl.loc[pl.Returning==True]
    pl['Season'] = y
    pl = pd.merge(pl, teams, how='left', on=['TeamID','TeamID'])
    pl = pl[['Season','TeamID','TeamName','pct_assist','pct_block','pct_cr_2pt','pct_made1_free','pct_made2_jump','pct_made3_jump','pct_rebound','pct_steal','pct_turnover']]
    
    if tl_df is not None:
        tl_df = pd.concat([tl_df,pl],axis=0)
    else:
        tl_df = pl
    
    

In [102]:
tl_df.to_csv(output+'TL_Returning.csv',index=None)


In [131]:
# uk = tl_df.loc[tl_df['TeamName'].isin(['Kentucky','Duke','North Carolina','Virginia','Louisville'])]
uk = tl_df.copy()

# schools = ['Kentucky','Duke','North Carolina','Virginia','Louisville']
# uk.loc[~uk['TeamName'].isin(schools), 'TeamName'] = 'Others'

uk['Season'] = uk['Season'].copy().astype(int)
# uk = uk.loc[uk.Season>=2014]
# uk = uk.sort_values(by=['TeamName','Season'])
uk = uk.groupby(['TeamName'])['pct_assist','pct_block','pct_cr_2pt','pct_made1_free','pct_made2_jump','pct_made3_jump','pct_rebound','pct_steal','pct_turnover'].mean().round(2).reset_index()

cols = ['TeamName','Assists','Blocks','CloseRange2pt','FreeThrows','2ptJump','3pt','Rebounds','Steals','Turnovers']
uk.columns = cols

uk.sort_values(by=['2ptJump']).head(6).reset_index(drop=True)


,TeamName,Assists,Blocks,CloseRange2pt,FreeThrows,2ptJump,3pt,Rebounds,Steals,Turnovers
0,Centenary,0.34,0.33,0.39,0.31,0.15,0.24,0.36,0.37,0.34
1,Kentucky,0.27,0.33,0.27,0.26,0.27,0.27,0.30,0.27,0.28
2,TX Southern,0.34,0.35,0.38,0.32,0.31,0.35,0.37,0.35,0.35
3,Florida Intl,0.37,0.46,0.40,0.38,0.33,0.41,0.40,0.41,0.39
4,Duke,0.45,0.46,0.42,0.40,0.37,0.43,0.44,0.46,0.43
5,Grambling,0.35,0.41,0.37,0.38,0.38,0.38,0.38,0.36,0.36
